In [24]:
import torch
from torch import nn
import torch.nn.functional as F
from torchinfo import summary
from torch_receptive_field import receptive_field

In [24]:
class LeNet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool1 = nn.AvgPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.AvgPool2d(2)
        self.linear1 = nn.Linear(5*5*16, 120)
        self.linear2 = nn.Linear(120, 84)
    def forward(self, x):
        out = torch.tanh(self.conv1(x))
        out = self.pool1(out)
        out = torch.tanh(self.conv2(out))
        out = self.pool2(out)
        out = torch.tanh(self.linear1(out.view(-1, 5*5*16)))
        out = self.linear2(out)
        return F.softmax(out, 1)

In [25]:
data = torch.ones([10, 1, 32, 32])

In [26]:
net = LeNet5()

In [27]:
out = net(data)

In [28]:
out.shape

torch.Size([10, 84])

# 定義網路

In [20]:
class LeNet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(1, 6, 5, stride=1, padding=0),
                                    nn.Tanh(),
                                    nn.AvgPool2d(kernel_size=2))
        self.layer2 = nn.Sequential(nn.Conv2d(6, 16, 5, stride=1, padding=0),
                                    nn.Tanh(),
                                    nn.AvgPool2d(kernel_size=2))
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 84)
    def forward(self, x):
        x = self.layer2(self.layer1(x))
        x = self.fc2(self.fc1(x.view(-1, 5*5*16)))
        return x

In [21]:
data = torch.ones([10, 1, 32, 32])
net = LeNet5()

In [22]:
net(data)

tensor([[ 0.0055,  0.1400,  0.0251, -0.0736, -0.1069,  0.0018,  0.0257,  0.1498,
          0.1021, -0.1240, -0.0666, -0.0454, -0.0477,  0.0913,  0.0416, -0.0606,
         -0.3317,  0.2084,  0.0902,  0.0520, -0.2106, -0.1463,  0.0869, -0.1817,
         -0.0293,  0.1044, -0.1731,  0.0256,  0.0551, -0.0459, -0.0020,  0.0538,
          0.0116, -0.0511, -0.0563, -0.1787, -0.0189,  0.1006, -0.0813,  0.2445,
          0.1333,  0.0630,  0.1106,  0.1370, -0.2684,  0.0480, -0.1608,  0.0794,
         -0.2318, -0.0668,  0.0099, -0.0453, -0.1230, -0.1458,  0.0293,  0.0828,
         -0.1610,  0.2832, -0.0418,  0.0299,  0.0100,  0.1055, -0.2054, -0.0515,
         -0.0098,  0.1421,  0.2088,  0.0185, -0.0567, -0.1526, -0.0366, -0.1263,
         -0.0009, -0.0888,  0.2147,  0.0716,  0.0083,  0.0939,  0.0492, -0.0914,
          0.0547, -0.0432, -0.0465,  0.0975],
        [ 0.0055,  0.1400,  0.0251, -0.0736, -0.1069,  0.0018,  0.0257,  0.1498,
          0.1021, -0.1240, -0.0666, -0.0454, -0.0477,  0.0913, 

In [23]:
summary(net, [10, 1, 32, 32], device="cpu")

Layer (type:depth-idx)                   Output Shape              Param #
LeNet5                                   [10, 84]                  --
├─Sequential: 1-1                        [10, 6, 14, 14]           --
│    └─Conv2d: 2-1                       [10, 6, 28, 28]           156
│    └─Tanh: 2-2                         [10, 6, 28, 28]           --
│    └─AvgPool2d: 2-3                    [10, 6, 14, 14]           --
├─Sequential: 1-2                        [10, 16, 5, 5]            --
│    └─Conv2d: 2-4                       [10, 16, 10, 10]          2,416
│    └─Tanh: 2-5                         [10, 16, 10, 10]          --
│    └─AvgPool2d: 2-6                    [10, 16, 5, 5]            --
├─Linear: 1-3                            [10, 120]                 48,120
├─Linear: 1-4                            [10, 84]                  10,164
Total params: 60,856
Trainable params: 60,856
Non-trainable params: 0
Total mult-adds (M): 4.22
Input size (MB): 0.04
Forward/backward pass siz

# 計算感受野

In [50]:
class LeNet5_receptive_field(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(1, 6, 5, stride=1, padding=0),  # 1 + (5-1) *1 = 5
                                    # nn.Tanh(),
                                    # nn.AvgPool2d(kernel_size=2)
                                    nn.MaxPool2d(kernel_size=2)  # 5 + (2-1) *1*1=6
                                   )
        self.layer2 = nn.Sequential(nn.Conv2d(6, 16, 5, stride=1, padding=0),  # 6 + (5-1) *2 = 14
                                    # nn.Tanh(),
                                    # nn.AvgPool2d(kernel_size=2)
                                    nn.MaxPool2d(kernel_size=2)  # 14 + (2-1)*2 = 16
                                   )
        # self.fc1 = nn.Linear(5*5*16, 120)
        # self.fc2 = nn.Linear(120, 84)
    def forward(self, x):
        x = self.layer2(self.layer1(x))
        # x = self.fc2(self.fc1(x.view(-1, 5*5*16)))
        return x

In [51]:
net_ = LeNet5_receptive_field()

In [52]:
# 套件僅支持架構內存在nn.Conv2d, nn.MaxPool2d

In [53]:
OrderedDict = receptive_field(net_.cuda(), (1, 32, 32))

------------------------------------------------------------------------------
        Layer (type)    map size      start       jump receptive_field 
        0               [32, 32]        0.5        1.0             1.0 
        1               [28, 28]        2.5        1.0             5.0 
        2               [14, 14]        3.0        2.0             6.0 
        3               [10, 10]        7.0        2.0            14.0 
        4                 [5, 5]        8.0        4.0            16.0 
